### Import libraries

In [ ]:
import requests,json,os
from datetime import date,datetime

### Load configuration file

In [ ]:
with open('configuration.txt') as json_file:
    config = json.load(json_file)
with open('credentials.txt') as json_file:
    credentials = json.load(json_file)
        

### Create URLs

In [ ]:
keys = "key="+credentials.get('api_key')+"&token="+credentials.get('api_token')
trello_base_url = "https://api.trello.com/1/"
board_url = trello_base_url+"boards/"+config.get('boardid')
url_cards = board_url+"/cards?attachments=true&customFieldItems=true&filter=all&"+keys
url_lists = board_url+"/lists?"+keys
url_customfields = board_url+"/customFields?"+keys
url_labels = board_url+"/labels?"+keys
url_members = board_url+"/members?"+keys


### Create object for statuses (WIP)

In [ ]:
statussen = ['Nog starten','Blocked','Doing','Done']

### Parse the JSON objects to Python objects

In [ ]:
cards = json.loads(json.dumps(requests.get(url_cards).json()))
lists = json.loads(json.dumps(requests.get(url_lists).json()))
customfields = json.loads(json.dumps(requests.get(url_customfields).json()))
labels = json.loads(json.dumps(requests.get(url_labels).json()))
members = json.loads(json.dumps(requests.get(url_members).json()))

### Create dictionary for custom fields

In [ ]:
customfields_dict = {}
for i in customfields:
    customfields_dict[i['id']] = {}
    if i['type'] == 'date':
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {'id': 'date'}
       

    else:
        customfields_dict[i['id']][i['name']] = {}
        customfields_dict[i['id']][i['name']]['options'] = {}
        for j in i['options']:
            customfields_dict[i['id']][i['name']]['options'][j['id']] =  j['value']['text']

customfieldsmetdate = []
for i,j in customfields_dict.items():
        for k,l in j.items():
            try:
                if l['options']['id'] == 'date':
                    customfieldsmetdate.append(i)
            except:
                pass

### Create object with all chosen lists

In [ ]:
chosenlists = []
for i in config.get('notstarted'):
    chosenlists.append(i)
chosenlists.extend(config.get('blocked'))
chosenlists.extend(config.get('doing'))
chosenlists.extend(config.get('done'))

### Create function to determine datetime of the card

In [ ]:
def idtodate(cardid):
    hex = cardid[0:8]
    timestamp = int(hex,16)
    timedate = datetime.fromtimestamp(timestamp)
    return timedate

### Create dictionary object for all cards

In [ ]:
kaarten = {}
for i in cards:
    kaarten[i['id']] = {'name': i['name'],
                        'cardid': i['id'],
                        'idlist': i['idList'],
                        'customfields': i['customFieldItems'],
                        'labels': {},
                        'members': {},
                        'sjabloon': i['isTemplate'],
                        'due': None,
                        'closed': i['closed'],
                        'attachments': {},
                        'shortUrl': i['shortUrl']
                       }
    for j in i['idMembers']:

        for k in members:

            if j == k['id']:
                    kaarten[i['id']]['members'][k['id']] = k['fullName']
    if i['due'] != None:
        kaarten[i['id']]['due'] = datetime.strptime(i['due'][0:19],'%Y-%m-%dT%H:%M:%S')
    for j in i['labels']:
        kaarten[i['id']]['labels'][j['id']] = j['name']
    for j in i['attachments']:
        try:
            if j['url'][0:21]== 'https://trello.com/c/':
                kaarten[i['id']]['attachments'][j['url'][21:29]] = None
        except:
            pass
                    
if customfields_dict != {}:
    for i,j in customfields_dict.items():
        for k,l in j.items():
            for m,n in kaarten.items():
                n[k] = None

    for i,j in kaarten.items():
        for k in j['customfields']:
            if k['idCustomField'] in customfieldsmetdate:
                for l,m in customfields_dict.items():
                    for n,o in m.items():
                        if k['idCustomField'] == l:
                            j[n] = datetime.strptime(k['value']['date'][0:19],'%Y-%m-%dT%H:%M:%S')
            else:
                for l,m in customfields_dict.items():
                    for n,o in m.items():
                        if k['idCustomField'] == l:
                            for p,q in o.items():
                                for r,s in q.items():
                                    if k['idValue'] == r:
                                        j[n] = s     





### Add status of the card

In [ ]:
for i,j in kaarten.items():
    date = idtodate(i)
    j['created'] = date
    for k in lists:
        if j['idlist'] == k['id']: j['list'] = k['name'] 
    if j['list'] in config.get('notstarted'):
        j['status'] = 'Not started'
    elif j['list'] in config.get('doing'):
        j['status'] = 'Doing'
    elif j['list'] in config.get('blocked'):
        j['status'] = 'Blocked'
    elif j['list'] in config.get('done'):
        j['status'] = 'Done'
    del j['customfields']
    del j['idlist']
    
# Give the status Archived if the card is closed and not done
for i,j in kaarten.items():
    if j['closed'] == True and j['status'] != 'Done':
        j['status'] = 'Archived'

### Delete unusable cards

In [ ]:
# Create object with lists that are not chosen
liststodelete = []
for i in lists:
    if i['name'] not in chosenlists:
        liststodelete.append(i['name'])

# Create object with all cards that should be deleted (ignored)
cardstodelete = []
for i,j in kaarten.items():
    if j['sjabloon'] == True:
        cardstodelete.append(i)
    elif j['list'] in liststodelete:
        cardstodelete.append(i)

# Delete the cards in the object 'cardstodelete'
for i in cardstodelete:
    if i in kaarten:
        del kaarten[i]
        
